In [390]:
from bs4 import BeautifulSoup
import requests

import unicodedata
from csv import writer
import re

import pandas as pd
import numpy as np

import json


#spark session
from pyspark.sql import SparkSession
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql import functions as F


import os
import re 

#folium
import geopandas as gpd
import re
import folium

In [391]:
# Create a spark session

spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .getOrCreate()
)

#read in the parquet
domain = spark.read.parquet('../data/raw/domain-website-data')


In [393]:
#drop NSW
domain = domain.filter(F.col('state') == 'VIC')

#drop column with too many missing values
domain = domain.drop('land_size','land_unit','is_rural','is_retirement','url').toPandas()

#filter property type, remove 'cat spcae' and 'Acreage / Semi-Rural'
selected_property_type = ['Apartment / Unit / Flat', 'Studio', 'Townhouse',
       'House', 'New House & Land', 'Semi-Detached', 'Villa', 'Terrace',
       'Acreage / Semi-Rural', 'New Apartments / Off the Plan', 'Duplex',
       'Unknown', 'Farm', 'Penthouse', 'Rural']

domain= domain.loc[domain['property_type'].isin(selected_property_type)]

In [394]:
domain

,property_id,street,suburb,state,postcode,latitude,longitude,price,bedrooms,bathrooms,parking,property_type
0,13671355,104/300 Swanston Street,MELBOURNE,VIC,3000,-37.810608,144.964584,$550,2,1,0,Apartment / Unit / Flat
1,14868152,101/25-33 Wills Street,MELBOURNE,VIC,3000,-37.810776,144.956924,$400,1,1,0,Apartment / Unit / Flat
2,15572564,2108/288 Spencer Street,MELBOURNE,VIC,3000,-37.813774,144.952087,$350,1,1,0,Apartment / Unit / Flat
3,15831027,2201/288 Spencer Street,MELBOURNE,VIC,3000,-37.813774,144.952087,$400,1,1,0,Apartment / Unit / Flat
4,15996004,502/5 Sutherland Street,MELBOURNE,VIC,3000,-37.811417,144.960846,$430 per week,1,1,0,Apartment / Unit / Flat
...,...,...,...,...,...,...,...,...,...,...,...,...
15253,16065590,5 Eurack Rd,BEEAC,VIC,3251,-38.171036,143.635600,UNDER APPLICATION,3,1,2,House
15254,16064656,7 Quigley St,YINNAR,VIC,3869,-38.324482,146.327637,$380 per week,3,2,2,House
15255,16062004,28 Monash Ave,OLINDA,VIC,3788,-37.853085,145.363983,$495 weekly,3,1,0,House
15256,15998256,Guyatts Road,TOONGABBIE,VIC,3856,-38.068634,146.615500,$600,4,2,2,House


In [ ]:
#remove instance with 'carpark' in street name

In [347]:
def get_rent(s):
    pattern = re.compile('^( )*([0-9]*)( )*')
    num = re.sub('(,)?', '',s)
    num = re.sub('[^0-9]+', ' ',num)
    match = re.match(pattern,num)[0].replace(" ", "")

    if match !='':
        return match
    else:
        return 0

In [408]:
domain['price_rent'] = [float(get_rent(i)) for i in domain['price']]
domain = domain[domain.price_rent > 40]

In [410]:
domain['price_rent'].describe()

count    1.484100e+04
mean     8.252461e+02
std      3.388904e+04
min      5.000000e+01
25%      3.850000e+02
50%      4.500000e+02
75%      5.650000e+02
max      4.125000e+06
Name: price_rent, dtype: float64